In [21]:
using DifferentialEquations, Statistics

In [2]:
struct CRMParams
    c::Matrix{Float64}
    e::Matrix{Float64}
    K::Vector{Float64}
    m::Vector{Float64}
    S::Int64
    M::Int64
    CRMParams(c,e,K,m) = size(c,1)==size(m,1) && size(c,2)==size(K,1) && size(c) == size(e) ? new(c,e,K,m,size(m)[1],size(K)[1]) : error("parameters wrong dimensions")
end

function CRM!(du,u,p,t)
    @inbounds λ = @view u[1:p.S]
    @inbounds dλ = @view du[1:p.S]
    @inbounds R = @view u[end+1-p.M:end]
    @inbounds dR = @view du[end+1-p.M:end]
    dλ .= λ .* (p.c * R .- p.m)
    dR .= R .* (p.K .- R .- p.e' * λ)
end

M = 200;
S = 200;
μc = 1e0M;
μe = 1e0M;
σc = 2e-1sqrt(M);
σe = 2e-1sqrt(M);
K = 1e0;
σK = 1e-1;
m = 1e0;
σm = 1e-1;

In [23]:
# function runCRM(ρ,tmax)
#     K0 = K .+ σK*randn(M);
#     params = randn(M,S) |> d -> CRMParams(μc/M .+ (σc/sqrt(M)).*d,μe/M .+ (σe/sqrt(M)).*(ρ.*d .+ sqrt(1-ρ^2).*randn(M,S)),K0,m .+ σm*randn(S))
#     u0=vcat(ones(S)/S,K0)
#     tspan=(0.0,tmax + 0.25);
#     prob = ODEProblem(CRM!,u0,tspan,params);
#     sol = solve(prob,abstol=1e-15,reltol=1e-15,saveat=(tmax-0.25):0.05:tmax+0.25);
#     return sol
# end
tmax=5000;
ρ=0.72;
K0 = K .+ σK*randn(M);
params = randn(M,S) |> d -> CRMParams(μc/M .+ (σc/sqrt(M)).*d,μe/M .+ (σe/sqrt(M)).*(ρ.*d .+ sqrt(1-ρ^2).*randn(M,S)),K0,m .+ σm*randn(S))
u0=vcat(ones(S)/S,K0)
tspan=(0.0,tmax + 0.25);
prob = ODEProblem(CRM!,u0,tspan,params);
solsave = solve(prob,abstol=1e-15,reltol=1e-15,saveat=(tmax-0.25):0.05:tmax+0.25);
sol = solve(prob,abstol=1e-15,reltol=1e-15);

In [24]:
tmax=5000;
ρ=0.72;
# K0 = K .+ σK*randn(M);
# params = randn(M,S) |> d -> CRMParams(μc/M .+ (σc/sqrt(M)).*d,μe/M .+ (σe/sqrt(M)).*(ρ.*d .+ sqrt(1-ρ^2).*randn(M,S)),K0,m .+ σm*randn(S))
u0=sol(tmax);
# u0=vcat(ones(S)/S,K0)
tspan=(0.0,tmax + 0.25);
prob = ODEProblem(CRM!,u0,tspan,params);
solsave = solve(prob,abstol=1e-15,reltol=1e-15,saveat=(tmax-0.25):0.05:tmax+0.25);
sol = solve(prob,abstol=1e-15,reltol=1e-15);

In [25]:
tmaxtotal=2*5000;
ρ=0.72;
# K0 = K .+ σK*randn(M);
# params = randn(M,S) |> d -> CRMParams(μc/M .+ (σc/sqrt(M)).*d,μe/M .+ (σe/sqrt(M)).*(ρ.*d .+ sqrt(1-ρ^2).*randn(M,S)),K0,m .+ σm*randn(S))
# u0=sol(tmaxtotal);
u0=vcat(ones(S)/S,K0)
tspan=(0.0,tmaxtotal + 0.25);
prob = ODEProblem(CRM!,u0,tspan,params);
solsavetotal = solve(prob,abstol=1e-15,reltol=1e-15,saveat=(tmaxtotal-0.25):0.05:tmaxtotal+0.25);
soltotal = solve(prob,abstol=1e-15,reltol=1e-15);

In [26]:
mean(abs2.(sol(tmax) - soltotal(tmaxtotal)))

2.3476018714383932e-33